In [50]:
import numpy as np
distances = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
classes = np.array([3,2,3,1,2,3,3,2,1])

In [51]:
# Common helper to handle tie-breaking
def handle_tie(classes, metric):
    return classes[np.argmax(metric)]  # Breaking ties by the largest metric

# distances: list of distances to the k nearest neighbours
# classes: list of classes of the k nearest neighbours
# class_weights: dictionary of class weights (optional)
def majority_class(distances, classes, class_weights=None):
    unique_classes, count = np.unique(classes, return_counts=True)

    # Apply class weights if provided
    if class_weights is not None:
        count = np.array([count[i] * class_weights.get(cls, 1) for i, cls in enumerate(unique_classes)])

    max_count = np.max(count)
    max_class = unique_classes[count == max_count]

    if len(max_class) == 1:
        return max_class[0]

    # Tie breaking by average distance
    avg_distances = np.array([np.mean(distances[classes == cls]) for cls in max_class])
    max_class = handle_tie(max_class, -avg_distances)

    return max_class


# distances: list of distances to the k nearest neighbours
# classes: list of classes of the k nearest neighbours
# class_weights: dictionary of class weights (optional)
def inverse_distance_weight(distances, classes, class_weights=None):
    unique_classes = np.unique(classes)
    metric = np.zeros(len(unique_classes))

    for i, cls in enumerate(unique_classes):
        d = distances[classes == cls]
        metric[i] = np.sum(1 / d)

    # Apply class weights if provided
    if class_weights is not None:
        metric = np.array([metric[i] * class_weights.get(cls, 1) for i, cls in enumerate(unique_classes)])

    max_count = np.max(metric)
    max_class = unique_classes[metric == max_count]

    if len(max_class) == 1:
        return max_class[0]

    # Tie breaking by the metric
    max_class = handle_tie(max_class, metric)
    return max_class


# distances: list of distances to the k nearest neighbours
# classes: list of classes of the k nearest neighbours
# class_weights: dictionary of class weights (optional)
def sheppards_work(distances, classes, class_weights=None):
    unique_classes = np.unique(classes)
    metric = np.zeros(len(unique_classes))

    for i, cls in enumerate(unique_classes):
        d = distances[classes == cls]
        metric[i] = np.sum(np.exp(-d))

    # Apply class weights if provided
    if class_weights is not None:
        metric = np.array([metric[i] * class_weights.get(cls, 1) for i, cls in enumerate(unique_classes)])

    max_count = np.max(metric)
    max_class = unique_classes[metric == max_count]

    if len(max_class) == 1:
        return max_class[0]

    # Tie breaking by the metric
    max_class = handle_tie(max_class, metric)
    return max_class

In [55]:
print(majority_class(distances, classes, {1: 0.01, 2: 10.0, 3: 0.5}))

2
